In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=6a87df14249b19b06394495305187b85d9ba1faf705187473a0747b86b2e1560
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, IntegerType, StructField, StructType
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# prompt: stock all the content of files in folder tp4_data at drive in a dataframe

conf = SparkConf().setAppName("tp3")
spark = SparkContext(conf=conf)

path = "/content/drive/My Drive/tp4_data/*.csv"
# Create a SQLContext object
sqlContext = SQLContext(spark)

# Read the CSV files into a DataFrame
df = sqlContext.read.csv(path, header=True, inferSchema=True)


/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [ ]:
df = df.na.fill(0)

In [ ]:
from pyspark.sql.functions import date_format

df = df.withColumn("day_of_week", date_format("InvoiceDate", "E"))


In [ ]:
df.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|   537226|    22811|SET OF 6 T-LIGHTS...|       6|2010-12-06 08:34:00|     2.95|   15987.0|United Kingdom|        Mon|
|   537226|    21713|CITRONELLA CANDLE...|       8|2010-12-06 08:34:00|      2.1|   15987.0|United Kingdom|        Mon|
|   537226|    22927|GREEN GIANT GARDE...|       2|2010-12-06 08:34:00|     5.95|   15987.0|United Kingdom|        Mon|
|   537226|    20802|SMALL GLASS SUNDA...|       6|2010-12-06 08:34:00|     1.65|   15987.0|United Kingdom|        Mon|
|   537226|    22052|VINTAGE CARAVAN G...|      25|2010-12-06 08:34:00|     0.42|   15987.0|United Kingdom|        Mon|
+---------+---------+-------------------

In [ ]:
train_df = df.filter(df.InvoiceDate < "2010-12-13")
test_df = df.filter(df.InvoiceDate >= "2010-12-13")


NameError: name 'df' is not defined

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="day_of_week", outputCol="day_of_week_index")
indexed_df = indexer.fit(train_df).transform(train_df)


In [ ]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCol="day_of_week_index", outputCol="day_of_week_encoded")
encoded_df = encoder.fit(indexed_df).transform(indexed_df)


In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=['UnitPrice', 'Quantity', 'day_of_week_encoded'],
    outputCol="features")

transformed_df = assembler.transform(encoded_df)


In [ ]:
from pyspark.ml import Pipeline

stages = [indexer, encoder, assembler]

pipeline = Pipeline(stages=stages)


In [ ]:
#Notre StringIndexer doit savoir combien de valeurs uniques il y a à indexer, comment résoudre ce problème ?
indexer = StringIndexer(inputCol="day_of_week", outputCol="day_of_week_index", handleInvalid="skip")

In [ ]:
pipeline_model = pipeline.fit(train_df)

transformed_df = pipeline_model.transform(train_df)

NameError: name 'pipeline' is not defined

In [ ]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(featuresCol='features', k=20)


In [ ]:
model = kmeans.fit(transformed_df)

In [ ]:
pipeline_test_model = pipeline.fit(test_df)

transformed_test_df = pipeline_model.transform(test_df)



In [ ]:
predictions = model.transform(transformed_test_df)


In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator()

# Evaluate the model on the test data
silhouette = evaluator.evaluate(predictions)

print("Silhouette coefficient:", silhouette)


ModuleNotFoundError: No module named 'pyspark'